#Importing Required libraries

In [ ]:
#Importng reuired libraries 
import imblearn
import pandas as pd
import numpy as np
import numpy as np
import collections
import warnings
from sklearn import metrics
from sklearn import tree
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,mutual_info_classif,chi2

In [1]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import imblearn 
import collections 
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer . 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelecKBest, mutual_info_classif, chi2

#Data preprocessing

In [ ]:
# loading data from the local directory 
training_df= pd.read_csv('/content/1667789589_795817_train.txt',delimiter = "\t", header = None)
testing_df=pd.read_csv('/content/1667789589_8086398_test.txt',delimiter = "\t", header = None)

In [ ]:
#training data 
training_df.columns=['Target','Features']
#testing data
testing_df.columns=['Features']

In [ ]:
#Splitting all the values in the dataframe 
training_x=training_df.Features.str.split(' ')
testing_x=testing_df.Features.str.split(' ')
training_y=training_df.Target

#Converting data to numeric

In [ ]:
#converting data to numeric for training and testing
training_x=training_x.apply(pd.to_numeric)
training_y=training_y.apply(pd.to_numeric)
testing_x=testing_x.apply(pd.to_numeric)

#Creating features

In [ ]:
# converting the data into one hot format
FeatureColumn=list(range(1,100001))
# For each data, 0 and 1 will based placed where if it is found in that row 
FeatureColumn = list(np.float_(FeatureColumn)) #converting data to float

In [ ]:
# appending for each features in the data (100001)
training_data=[]
for i in training_x: #iterating in training data 
    frequency = collections.Counter(i)  #calculating frequency of each vlaues in row
    training_data.append(frequency) #Appending dictinary values to training data 

In [ ]:
#creating dataframe based on frequency of each values
x_training = pd.DataFrame(training_data,columns=FeatureColumn)

In [ ]:
#getting all the frequency values for each testing data
testing_data=[]
for i in testing_x: #iterating in testing data 
    frequency = collections.Counter(i)
    testing_data.append(frequency)

In [ ]:
#creating dataframe for testing data based on frequency of each values
x_testing = pd.DataFrame(testing_data,columns =FeatureColumn)

#Handling Null values 

In [ ]:
#simple imputer for removing null or missing values from the dataset
MissingValueImputer = SimpleImputer(missing_values = np.nan,strategy ='constant')

In [ ]:
#Treating null values for training and testing data 
MissngTraining = MissingValueImputer.fit_transform(x_training)
MissngTesting = MissingValueImputer.transform(x_testing)

#Data Splitting 

In [ ]:
#Splitting dataset 
X_training, X_testing, y_training, y_testing = train_test_split(MissngTraining,training_y,test_size=0.33,random_state=42)

#Feature Selection 

In [ ]:
FeatureSelection=SelectKBest(chi2, k=300)

In [ ]:
Feature_training=FeatureSelection.fit_transform(X_training,y_training)
Feature_testing=FeatureSelection.transform(X_testing)

#Using DataBalanced function for resolving unbalanced data problems

In [ ]:
#Data balance
DataBalanced = SMOTE()
x_DataBalanced, y_DataBalanced = DataBalanced.fit_resample(Feature_training,y_training)

#Boosting classifier 

In [ ]:
class BoostingClassifier:
    def __init__(self):
        self.nodes = None
        self.Weights = None
        self.ErrorRate = None
        self.SampleWeights = None

    def TargetValues(self, TrainingFeatures, y):
        assert set(y) == {1, 0}
        return TrainingFeatures, y

def FitMethod(self, TrainingFeatures, y, iters):
    TrainingFeatures, y = self.TargetValues(TrainingFeatures, y)
    n = TrainingFeatures.shape[0]
    self.SampleWeights = np.zeros(shape=(iters, n))
    self.nodes = np.zeros(shape=iters, dtype=object)
    self.Weights = np.zeros(shape=iters)
    self.ErrorRate = np.zeros(shape=iters)

    self.SampleWeights[0] = np.ones(shape=n) / n
    for t in range(iters):
        CurrentWeights = self.SampleWeights[t]
        ADANodes = DecisionTreeClassifier(max_depth=1, max_leaf_nodes=2)
        ADANodes = ADANodes.fit(TrainingFeatures, y, sample_weight=CurrentWeights)
        ADANodes_pred = ADANodes.predict(TrainingFeatures)
        err = CurrentWeights[(ADANodes_pred != y)].sum()# / n
        ADANodes_weight = np.log((1 - err) / err) / 2
        calculated_weights_ada = CurrentWeights * np.exp(-ADANodes_weight * y * ADANodes_pred)
        calculated_weights_ada /= calculated_weights_ada.sum()
        if t+1 < iters:
            self.SampleWeights[t+1] = calculated_weights_ada

        self.nodes[t] = ADANodes
        self.Weights[t] = ADANodes_weight
        self.ErrorRate[t] = err
    return self

#Model prediction and Performance 

In [ ]:
def predict(self, TrainingFeatures):
    pred_ada = np.array([ADANodes.predict(TrainingFeatures) for ADANodes in self.nodes])
    return np.sign(np.dot(self.Weights, pred_ada))

In [ ]:
BoostingClassifier.FitMethod = FitMethod
BoostingClassifier.predict = predict
ada_classfier = BoostingClassifier().FitMethod(x_DataBalanced, y_DataBalanced, iters=11)

In [ ]:
y_pred = ada_classfier.predict(Feature_testing)
accuracy_score(y_testing, y_pred)

0.9583333333333334

#Result/Output

In [ ]:
TrainingAfterImputer=FeatureSelection.fit_transform(MissngTraining,training_y)
TestingAfterImputer=FeatureSelection.transform(MissngTesting)

In [ ]:
x_DataBalanced, y_DataBalanced = DataBalanced.fit_resample(TrainingAfterImputer,training_y)

In [ ]:
clf = BoostingClassifier().FitMethod(x_DataBalanced, y_DataBalanced, iters=11)
y_pre = clf.predict(TestingAfterImputer)

#Testing

In [ ]:
ans=pd.read_csv("/content/1667789589_8151922_formatfile.txt",header = None)

In [ ]:
testing_data=np.array(ans[0])

In [ ]:
accuracy_score(testing_data, y_pre)

0.8

In [ ]:
#Saving output file 
with open("Answer.txt", "w") as file:
    for pred in y_pre:
      file.write(str(int(pred))+'\n')